In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
import shutil
from pyspark.sql.functions import col, when
from pyspark.sql.types import StructType,StructField,DoubleType, StringType, IntegerType
from pyspark import SparkConf
from pyspark.sql import SparkSession

启动MySQL服务
service mysql start
查看MySQL服务状态
service mysql status
连接MySQL数据库（首次连接）
mysql -S /var/run/mysqld/mysqld.sock -uroot
后续连接MySQL数据库
mysql -uroot -p

In [ ]:
创建数据库：
create database Survival_Analysis

In [3]:
conf = SparkConf().setAppName("Spark Read MySQL").set("spark.jars", "/data/lab/mysql-connector-j-8.4.0.jar") .set("spark.ui.port", "4058")
spark = SparkSession.builder.config(conf=conf).getOrCreate()


25/04/07 10:41:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/07 10:41:32 WARN Utils: Service 'SparkUI' could not bind on port 4058. Attempting port 4059.
25/04/07 10:41:32 WARN Utils: Service 'SparkUI' could not bind on port 4059. Attempting port 4060.
25/04/07 10:41:32 WARN Utils: Service 'SparkUI' could not bind on port 4060. Attempting port 4061.
25/04/07 10:41:32 WARN Utils: Service 'SparkUI' could not bind on port 4061. Attempting port 4062.


In [4]:
# Define schema
schema = StructType([
  StructField('customerID', StringType()),
  StructField('gender', StringType()),
  StructField('seniorCitizen', DoubleType()),
  StructField('partner', StringType()),
  StructField('dependents', StringType()),
  StructField('tenure', DoubleType()),
  StructField('phoneService', StringType()),
  StructField('multipleLines', StringType()),
  StructField('internetService', StringType()), 
  StructField('onlineSecurity', StringType()),
  StructField('onlineBackup', StringType()),
  StructField('deviceProtection', StringType()),
  StructField('techSupport', StringType()),
  StructField('streamingTV', StringType()),
  StructField('streamingMovies', StringType()),
  StructField('contract', StringType()),
  StructField('paperlessBilling', StringType()),
  StructField('paymentMethod', StringType()),
  StructField('monthlyCharges', DoubleType()),
  StructField('totalCharges', DoubleType()),
  StructField('churn', StringType())
  ])


In [4]:
#读取数据
bronze_df = spark.read.csv('Telco-Customer-Churn.csv', header=True, schema=schema)


In [7]:
#进行数据清洗，转换数据
silver_df = bronze_df.withColumn('churn',when(col('churn') == 'Yes',1).when(col('churn') == 'No',0).otherwise('Unknown'))\
                     .filter(col('contract') == 'Month-to-month')\
                     .filter(col('internetService') != 'No')


In [10]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark import SparkConf
from pyspark.sql import SparkSession

# data =result_df
# df = spark.createDataFrame(data, schema)
bronze_df.write.format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/Survival_Analysis?useSSL=false&serverTimezone=UTC") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", "bronze_df") \
        .option("user", "root") \
        .option("password", "123456") \
        .mode("append") \
        .save()

silver_df.write.format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/Survival_Analysis?useSSL=false&serverTimezone=UTC") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", "silver_df") \
        .option("user", "root") \
        .option("password", "123456") \
        .mode("append") \
        .save()

In [17]:
jdbc_df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|seniorCitizen|partner|dependents|tenure|phoneService|   multipleLines|internetService|     onlineSecurity|       onlineBackup|   deviceProtection|        techSupport|        streamingTV|    streamingMovies|      contract|paperlessBilling|       paymentMethod|monthlyCharges|totalCharges|churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|          0.0|    Yes|        No|   1.0|  

In [7]:
jdbc_df = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/Survival_Analysis?useSSL=false&serverTimezone=UTC") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "bronze_df") \
    .option("user", "root") \
    .option("password", "123456") \
    .load()
jdbc_df.createOrReplaceTempView("j")  # 注册为名为 "jdbc_table" 的临时视图
jdbc_df.printSchema()


root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- seniorCitizen: double (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- tenure: double (nullable = true)
 |-- phoneService: string (nullable = true)
 |-- multipleLines: string (nullable = true)
 |-- internetService: string (nullable = true)
 |-- onlineSecurity: string (nullable = true)
 |-- onlineBackup: string (nullable = true)
 |-- deviceProtection: string (nullable = true)
 |-- techSupport: string (nullable = true)
 |-- streamingTV: string (nullable = true)
 |-- streamingMovies: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessBilling: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- monthlyCharges: double (nullable = true)
 |-- totalCharges: double (nullable = true)
 |-- churn: string (nullable = true)



下列是几个deepdeek生成的sql查询语句，提示词为：
你是一个只能SQL生成器，我这里有一个名叫j的表，结构为：
|-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- seniorCitizen: double (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- tenure: double (nullable = true)
 |-- phoneService: string (nullable = true)
 |-- multipleLines: string (nullable = true)
 |-- internetService: string (nullable = true)
 |-- onlineSecurity: string (nullable = true)
 |-- onlineBackup: string (nullable = true)
 |-- deviceProtection: string (nullable = true)
 |-- techSupport: string (nullable = true)
 |-- streamingTV: string (nullable = true)
 |-- streamingMovies: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessBilling: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- monthlyCharges: double (nullable = true)
 |-- totalCharges: double (nullable = true)
 |-- churn: string (nullable = true)
 请生成一些有趣的查询，直接给出sql语句即可

In [10]:

spark.sql( """SELECT
customerID, tenure, monthlyCharges, totalCharges 
FROM j WHERE churn = 'Yes' AND totalCharges > (SELECT AVG(totalCharges) FROM j)""").show()

+----------+------+--------------+------------+
|customerID|tenure|monthlyCharges|totalCharges|
+----------+------+--------------+------------+
|7892-POOKP|  28.0|         104.8|     3046.05|
|0280-XJGEX|  49.0|         103.7|      5036.3|
|6467-CHFZW|  47.0|         99.35|     4749.15|
|5380-WJKOV|  34.0|        106.35|     3549.25|
|4598-XLKNJ|  25.0|          98.5|      2514.5|
|3192-NQECA|  68.0|         110.0|     7611.85|
|0486-HECZI|  55.0|         96.75|      5238.9|
|4846-WHAFZ|  37.0|          76.5|     2868.15|
|3445-HXXGF|  58.0|          45.3|      2651.2|
|0691-JVSYA|  53.0|         94.85|      5000.2|
|4484-GLZOU|  52.0|        105.05|     5624.85|
|0265-EDXBD|  32.0|          74.9|     2303.35|
|2321-OMBXY|  38.0|          80.3|     3058.65|
|5020-ZSTTY|  54.0|         82.45|      4350.1|
|4322-RCYMT|  68.0|          75.8|     5293.95|
|6680-NENYN|  43.0|         104.6|     4759.85|
|1343-EHPYB|  63.0|         103.4|      6603.0|
|9601-BRXPO|  25.0|        104.95|      

In [12]:
spark.sql("""
SELECT internetService, COUNT(*) AS senior_count 
FROM j 
WHERE seniorCitizen = 1 
GROUP BY internetService 
ORDER BY senior_count DESC
""").show()

+---------------+------------+
|internetService|senior_count|
+---------------+------------+
|    Fiber optic|        1662|
|            DSL|         518|
|             No|         104|
+---------------+------------+



In [14]:
spark.sql("""
SELECT contract, 
       AVG(tenure) AS avg_tenure,
       AVG(CASE WHEN churn = 'Yes' THEN 1.0 ELSE 0 END) AS churn_rate
FROM j 
WHERE dependents = 'Yes' AND partner = 'Yes'
GROUP BY contract
""").show()

+--------------+-----------------+----------+
|      contract|       avg_tenure|churn_rate|
+--------------+-----------------+----------+
|Month-to-month| 23.3213058419244|   0.32818|
|      One year|42.21748878923767|   0.09417|
|      Two year|55.40083217753121|   0.02219|
+--------------+-----------------+----------+



In [16]:
spark.sql("""
SELECT onlineSecurity, onlineBackup, techSupport,
       COUNT(*) AS user_count,
       AVG(monthlyCharges) AS avg_charge
FROM j 
WHERE internetService = 'Fiber optic'
GROUP BY onlineSecurity, onlineBackup, techSupport
""").show()

+--------------+------------+-----------+----------+------------------+
|onlineSecurity|onlineBackup|techSupport|user_count|        avg_charge|
+--------------+------------+-----------+----------+------------------+
|           Yes|         Yes|         No|       460| 98.29195652173914|
|            No|         Yes|        Yes|       506|101.19426877470356|
|           Yes|         Yes|        Yes|       518|107.97200772200775|
|           Yes|          No|        Yes|       230|100.68347826086955|
|            No|          No|         No|      2328| 82.72933848797256|
|            No|          No|        Yes|       478|  94.0271966527197|
|            No|         Yes|         No|      1202| 92.62362728785355|
|           Yes|          No|         No|       470| 89.76808510638303|
+--------------+------------+-----------+----------+------------------+



In [ ]:
接下来是两条deepseek生成错误的sql语句以及答案:

第一条的提示词：
|-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- seniorCitizen: double (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- tenure: double (nullable = true)
 |-- phoneService: string (nullable = true)
 |-- multipleLines: string (nullable = true)
 |-- internetService: string (nullable = true)
 |-- onlineSecurity: string (nullable = true)
 |-- onlineBackup: string (nullable = true)
 |-- deviceProtection: string (nullable = true)
 |-- techSupport: string (nullable = true)
 |-- streamingTV: string (nullable = true)
 |-- streamingMovies: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessBilling: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- monthlyCharges: double (nullable = true)
 |-- totalCharges: double (nullable = true)
 |-- churn: string (nullable = true)根据这个表，表名为j,根据性别分组查询留存率

In [18]:
#错误的
spark.sql("""
SELECT 
    gender AS 性别,
    COUNT(*) AS 总用户数,
    SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS 留存用户数,
    ROUND(
        AVG(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) * 100, 
        2
    ) AS 留存率百分比
FROM j
GROUP BY gender;
""").show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '性'.(line 3, pos 14)

== SQL ==

SELECT 
    gender AS 性别,
--------------^^^
    COUNT(*) AS 总用户数,
    SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS 留存用户数,
    ROUND(
        AVG(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) * 100, 
        2
    ) AS 留存率百分比
FROM j
GROUP BY gender;


In [22]:
#正确的
spark.sql("""
SELECT 
    gender AS `性别`,
    COUNT(*) AS `总用户数`,
    SUM(CASE WHEN LOWER(TRIM(churn)) = 'no' THEN 1 ELSE 0 END) AS `留存用户数`,
    CONCAT(  -- 使用 CONCAT 拼接百分比符号
        ROUND(
            (SUM(CASE WHEN LOWER(TRIM(churn)) = 'no' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 
            2
        ),
        '%'
    ) AS `留存率百分比`  -- 列名保持不变，值末尾添加 %
FROM j
WHERE gender IS NOT NULL
GROUP BY gender
""").show()

+------+--------+----------+------------+
|  性别|总用户数|留存用户数|留存率百分比|
+------+--------+----------+------------+
|Female|    6976|      5098|      73.08%|
|  Male|    7110|      5250|      73.84%|
+------+--------+----------+------------+



原因：
AI的语言模型基于训练数据中的常见模式生成代码，在大量中文技术文档和中文注释的代码示例影响下，容易混淆自然语言描述和实际代码规范。这种"语言混用幻觉"本质上是统计模型在代码生成场景下的典型局限，无法真正理解不同数据库系统对标识符的具体处理规则。

In [37]:
#错误的
spark.sql("""
-- 步骤1: 动态生成斐波那契数列（前20项）
WITH RECURSIVE fib_sequence AS (
  SELECT 
    0 AS n, 
    1 AS fib_prev, 
    1 AS fib_current  -- 初始化斐波那契数列前两项
  UNION ALL
  SELECT 
    n + 1,
    fib_current, 
    fib_prev + fib_current
  FROM fib_sequence
  WHERE n < 19  -- 生成前20项（覆盖tenure最大值）
),
fib_numbers AS (
  SELECT DISTINCT fib_prev AS fib FROM fib_sequence
  UNION 
  SELECT 1  -- 补充初始值1
),

-- 步骤2: 创建动态分箱区间（左闭右开）
fib_bins AS (
  SELECT 
    fib AS lower,
    COALESCE(
      LEAD(fib) OVER (ORDER BY fib), 
      999999  -- 最后一个分箱上限设为极大值
    ) AS upper
  FROM fib_numbers
  WHERE fib > 0
),

-- 步骤3: 用户分箱及留存率计算
user_bins AS (
  SELECT 
    j.*,
    CONCAT(fb.lower, '-', fb.upper) AS tenure_bin
  FROM j
  INNER JOIN fib_bins fb 
    ON j.tenure >= fb.lower 
    AND j.tenure < fb.upper
)

-- 最终结果：分箱留存率统计
SELECT 
  tenure_bin,
  COUNT(*) AS total_users,
  SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS retained_users,
  ROUND(
    SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) / COUNT(*), 
    2
  ) AS retention_rate
FROM user_bins
GROUP BY tenure_bin
ORDER BY MIN(fb.lower);
""").show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'fib_sequence'.(line 3, pos 15)

== SQL ==

-- 步骤1: 动态生成斐波那契数列（前20项）
WITH RECURSIVE fib_sequence AS (
---------------^^^
  SELECT 
    0 AS n, 
    1 AS fib_prev, 
    1 AS fib_current  -- 初始化斐波那契数列前两项
  UNION ALL
  SELECT 
    n + 1,
    fib_current, 
    fib_prev + fib_current
  FROM fib_sequence
  WHERE n < 19  -- 生成前20项（覆盖tenure最大值）
),
fib_numbers AS (
  SELECT DISTINCT fib_prev AS fib FROM fib_sequence
  UNION 
  SELECT 1  -- 补充初始值1
),

-- 步骤2: 创建动态分箱区间（左闭右开）
fib_bins AS (
  SELECT 
    fib AS lower,
    COALESCE(
      LEAD(fib) OVER (ORDER BY fib), 
      999999  -- 最后一个分箱上限设为极大值
    ) AS upper
  FROM fib_numbers
  WHERE fib > 0
),

-- 步骤3: 用户分箱及留存率计算
user_bins AS (
  SELECT 
    j.*,
    CONCAT(fb.lower, '-', fb.upper) AS tenure_bin
  FROM j
  INNER JOIN fib_bins fb 
    ON j.tenure >= fb.lower 
    AND j.tenure < fb.upper
)

-- 最终结果：分箱留存率统计
SELECT 
  tenure_bin,
  COUNT(*) AS total_users,
  SUM(CASE WHEN churn = 'No

这是第二条的提示词：
|-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- seniorCitizen: double (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- tenure: double (nullable = true)
 |-- phoneService: string (nullable = true)
 |-- multipleLines: string (nullable = true)
 |-- internetService: string (nullable = true)
 |-- onlineSecurity: string (nullable = true)
 |-- onlineBackup: string (nullable = true)
 |-- deviceProtection: string (nullable = true)
 |-- techSupport: string (nullable = true)
 |-- streamingTV: string (nullable = true)
 |-- streamingMovies: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessBilling: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- monthlyCharges: double (nullable = true)
 |-- totalCharges: double (nullable = true)
 |-- churn: string (nullable = true)根据这个表，表名为j,将tenure按斐波那契数列分箱（1,2,3,5,8...），统计每个分箱的留存率，要求动态生成分箱标签

In [39]:
#正确的
spark.sql("""

-- 步骤1: 使用数学公式生成斐波那契数列（前20项）
WITH fib_numbers AS (
  SELECT 
    CAST(
      (POW((1 + SQRT(5)) / 2, n) - POW((1 - SQRT(5)) / 2, n)) / SQRT(5) 
      AS INT
    ) AS fib
  FROM (
    SELECT explode(sequence(1, 20)) AS n  -- 生成20项足够覆盖常见tenure范围
  )
),
fib_distinct AS (
  SELECT DISTINCT fib 
  FROM fib_numbers 
  WHERE fib > 0
  UNION SELECT 1  -- 补充缺失的初始值1
),

-- 步骤2: 创建分箱区间（左闭右开）
fib_bins AS (
  SELECT 
    fib AS lower,
    COALESCE(
      LEAD(fib) OVER (ORDER BY fib), 
      999999  -- 最后一个分箱上限设为极大值
    ) AS upper
  FROM fib_distinct
),

-- 步骤3: 用户分箱及留存率计算
user_bins AS (
  SELECT 
    j.*,
    CONCAT(fb.lower, '-', fb.upper) AS tenure_bin
  FROM j
  INNER JOIN fib_bins fb 
    ON j.tenure >= fb.lower 
    AND j.tenure < fb.upper
  WHERE j.tenure IS NOT NULL  -- 排除空值
)

-- 最终结果：分箱留存率统计
SELECT 
  tenure_bin,
  COUNT(*) AS total_users,
  SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS retained_users,
  ROUND(
    SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 
    2
  ) AS retention_rate
FROM user_bins
GROUP BY tenure_bin
ORDER BY tenure_bin;  -- 按分箱标签排序
""").show()

25/04/07 12:23:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/07 12:23:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/07 12:23:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/07 12:23:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/07 12:23:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/07 12:23:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/07 1

+----------+-----------+--------------+--------------+
|tenure_bin|total_users|retained_users|retention_rate|
+----------+-----------+--------------+--------------+
|       1-2|       1226|           466|          0.38|
|     13-21|       1384|           956|          0.69|
|       2-3|        476|           230|          0.48|
|     21-34|       1922|          1476|          0.77|
|       3-5|        752|           398|          0.53|
|     34-55|       2770|          2270|          0.82|
|       5-8|        748|           438|          0.59|
|     55-89|       3638|          3348|          0.92|
|      8-13|       1148|           744|          0.65|
+----------+-----------+--------------+--------------+



原因：面对复杂的sql语句,deepseek会出现知识混杂现象，比如在 Spark SQL 中，递归 CTE（WITH RECURSIVE）尚未被支持，导致语法解析错误。

my assumptions that when the LLMs might fail in performing data analysis using SQL:
1.提示词中表的结构没有明确给出，llm 会瞎编乱造列名
2. AI可能混淆方言导致语法错误
3. 保留字未转义，不同数据库的保留字等不一样
4. AI的语言模型基于训练数据中的常见模式生成代码，在大量中文技术文档和中文注释的代码示例影响下，容易混淆自然语言描述和实际代码规范。
5 .AI可能在生成JOIN时没有正确指定ON子句，导致笛卡尔积或语法错误。比如，忘记添加连接条件，或者错误地使用逗号分隔表而没有JOIN关键字。